In [1]:
# import modules
from datetime import datetime, tzinfo, timedelta
#import hdmf._version
from dateutil.tz import tzlocal
import platform
import math
import numpy as np
import uuid
import os
from os import environ
import scipy.io as sio
#from hdmf.backends.hdf5.h5_utils import H5DataIO
import pandas as pd

from pynwb import NWBHDF5IO

#https://pypi.org/project/pyabf/
import pyabf

import pynwb
print('Using pynwb v%s'%pynwb.__version__)

Using pynwb v2.3.2


In [72]:
from nwbwidgets.timeseries import SeparateTracesPlotlyWidget,AlignMultiTraceTimeSeriesByTrialsVariable

In [73]:
from nwbwidgets.timeseries import MultiTimeSeriesWidget,AlignMultiTraceTimeSeriesByTrialsAbstract

In [4]:
import concurrent.futures
from pathlib import Path

import fsspec
import h5py
import ipywidgets as widgets
from dandi.dandiapi import DandiAPIClient
from fsspec.implementations.cached import CachingFileSystem
from pynwb import NWBHDF5IO
from tqdm.notebook import tqdm

from nwbwidgets.utils.dandi import (
    get_dandiset_metadata,
    get_file_url,
    has_nwb,
    list_dandiset_files,
)

In [5]:
from IPython.display import display

import numpy as np
import plotly.graph_objects as go
from ipywidgets import Layout, widgets
from tifffile import imread

from nwbwidgets.controllers import StartAndDurationController
from nwbwidgets.image import ImageSeriesWidget
from nwbwidgets.timeseries import SingleTracePlotlyWidget
from nwbwidgets.utils.timeseries import get_timeseries_maxt, get_timeseries_mint

from nwbwidgets.panel import Panel

In [6]:
class Dashboard2(Panel):
    def __init__(self,stream_mode: str = "fsspec",
        enable_dandi_source: bool = True,
        enable_s3_source: bool = True,
        enable_local_source: bool = True,
        **kwargs
    ):
        
        super().__init__(self,stream_mode,
        enable_dandi_source,
        enable_s3_source,
        enable_local_source,
        **kwargs)
    
    def stream_dandiset_file(self, args=None):
        """Stream NWB file from DANDI"""
        print("enterstream")
        self.widgets_panel.children = [widgets.Label("loading...")]
        dandiset_id = self.source_dandi_id.value.split("-")[0].strip()
        file_path = self.source_dandi_file_dropdown.value
        s3_url = get_file_url(dandiset_id=dandiset_id, file_path=file_path)
        self._stream_s3_file(s3_url)

    def _stream_s3_file(self, s3_url):
        print("enters3file")
        if self.stream_mode == "ros3":
            print("enteros3")
            io = NWBHDF5IO(s3_url, mode="r", load_namespaces=True, driver="ros3")

        else :
            print("enterfsspec")
            fs = fsspec.filesystem("http")
            f = fs.open(s3_url, "rb")
            file = h5py.File(f)
            io = NWBHDF5IO(file=file, load_namespaces=True)

        self.nwb = io.read()
        self.process() 
        
    
    def load_local_dir_file(self, args=None):
        """Load local NWB file"""
        full_file_path = str(Path(self.local_dir_path.value) / self.local_dir_files.value)
        io = NWBHDF5IO(full_file_path, mode="r", load_namespaces=True)
        self.nwb = io.read()
        self.process()
        
    
    def load_local_file(self, args=None):
        """Load local NWB file"""
        full_file_path = str(Path(self.local_file_path.value))
        io = NWBHDF5IO(full_file_path, mode="r", load_namespaces=True)
        self.nwb = io.read()
        self.process()
        
    def process(self):
        
        self.acqkeys = list(self.nwb.acquisition.keys())
        self.stimkeys = list(self.nwb.stimulus.keys())
        
        in1 = 0
        
        #Start time and duration controller
        self.tmin = get_timeseries_mint(
            self.make_ts(self.nwb.acquisition[self.acqkeys[in1]])
        )
        self.tmax = get_timeseries_maxt(
            self.make_ts(self.nwb.acquisition[self.acqkeys[in1]])
        )
        self.time_window_controller = StartAndDurationController(
            tmin=self.tmin,
            tmax=self.tmax,
            start=0
        )
        
        self.colorList = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
        
                # Electrophys single trace 
        self.electrical = SingleTracePlotlyWidget(
            timeseries=self.make_ts(self.nwb.acquisition[self.acqkeys[in1]]),
                              foreign_time_window_controller=self.time_window_controller)
        self.ebase = self.electrical.out_fig.data[0].x
        
        for i in range(1,len(self.acqkeys)):
            self.electrical.out_fig.add_trace(go.Scatter(x=self.ebase,y=self.nwb.acquisition[self.acqkeys[i]].data[:],line=dict(color=self.colorList[i%10])))
        
        
        self.electrical.out_fig.update_layout(
            title=None,
            showlegend=False,
            xaxis_title=None,
            yaxis_title="mVolts",
            width=840,
            height=230,
            margin=dict(l=60, r=200, t=8, b=20),
            # yaxis={"position": 0, "anchor": "free"},
            yaxis={
                "range": [
                     min(self.electrical.out_fig.data[0].y),
                     max(self.electrical.out_fig.data[-1].y),
                ],
                "autorange": False,
            },
            xaxis={
                "showticklabels": False,
                "ticks": "",
                "range": [
                    min(self.electrical.out_fig.data[0].x),
                    max(self.electrical.out_fig.data[0].x),
                ],
                "autorange": False,
            },
        )
        
        
        
        # Fluorescence single trace
        self.fluorescence = SingleTracePlotlyWidget(
            timeseries=self.make_ts(self.nwb.stimulus[self.stimkeys[in1]]),
                              foreign_time_window_controller=self.time_window_controller)
        
        self.fbase = self.fluorescence.out_fig.data[0].x
        
        for i in range(1,len(self.stimkeys)):
            self.fluorescence.out_fig.add_trace(go.Scatter(x=self.fbase,y=self.nwb.stimulus[self.stimkeys[i]].data[:],line=dict(color=self.colorList[i%10])))
            
        
        self.fluorescence.out_fig.update_layout(
            title=None,
            showlegend=False,
            width=840,
            height=230,
            margin=dict(l=60, r=200, t=8, b=20),
            yaxis_title="mA",
            yaxis={
                "range": [
                    min(self.fluorescence.out_fig.data[0].y),
                    max(self.fluorescence.out_fig.data[-1].y),
                ],
                "autorange": False,
            },
            xaxis={
                "range": [
                    min(self.fluorescence.out_fig.data[0].x),
                    max(self.fluorescence.out_fig.data[0].x),
                ],
                "autorange": False,
                "constrain": "domain",
                "anchor": "free",
            },
        )
        
        self.trace_widgets = self.makeTraceWid()
        
        self.tvaluelist = [False for i in range(len(self.tracelist))]
        self.oldtval = self.tvaluelist
        
        for i in range(len(self.tracelist)):
            self.tracelist[i].observe(self.obs)
                
        hbox_header = widgets.HBox([self.time_window_controller])
        vbox_widgets = widgets.VBox([self.electrical, self.fluorescence])
        hbox_widgets = widgets.HBox([vbox_widgets])
        hbox_widgets2 = widgets.HBox([self.trace_widgets])
        
        self.widgets_panel.children = [hbox_header, hbox_widgets, hbox_widgets2]
        
 
    def process_dandiset(self, dandiset):
        try:
            metadata = dandiset.get_metadata()
            if has_nwb(metadata):
                return metadata
        except:
            pass
        return None       


    def toggle_cache(self, args):
        if isinstance(args["new"], dict):
            if args["new"].get("value") is True:
                self.cache_path_text.layout.visibility = "visible"
            elif args["new"].get("value") is False:
                self.cache_path_text.layout.visibility = "hidden"
     
    def make_ts(self,nwb_obj,name="Trace"):
        return pynwb.base.TimeSeries(name=name,data=nwb_obj.data[:],starting_time=0.,rate=nwb_obj.rate,unit=nwb_obj.starting_time_unit)
    

    
    def updateVals(self):
        self.tvaluelist = [self.tracelist[i].value for i in range(len(self.tracelist))]
    
    def makeTraceWid(self):
        self.tracelist = []
        for i in range(len(self.stimkeys)):
            self.tracelist.append(widgets.ToggleButton( value=False, description=str(i), disabled=False, button_style='', # 'success', 'info', 'warning', 'danger' or '' 
                                                         tooltip='Description', icon='',color='red',background_color='yellow',width='30px'
                                                        ))
            self.tracelist[i].style.text_color = self.colorList[i%10]
                
        return(widgets.GridBox(self.tracelist, layout=widgets.Layout(grid_template_columns="repeat(4, 150px)")))
    
    def obs(self,change):
        if isinstance(change["new"], bool):
            self.updateVals()
            print(self.tvaluelist)
            self.compute_plot()
            
    def compute_plot(self):
        if(self.tvaluelist == [False for i in range(len(self.tvaluelist))]):
            for i in range(len(self.tvaluelist)):
                    self.electrical.out_fig.data[i].visible = True
                    self.fluorescence.out_fig.data[i].visible = True
        else:
            for i in range(len(self.tvaluelist)):
                self.electrical.out_fig.data[i].visible = self.tvaluelist[i]
                self.fluorescence.out_fig.data[i].visible = self.tvaluelist[i]
            
        
        
        
        
        
        
    

In [7]:
app = Dashboard2("fsspec")
app

Dashboard2(children=(HBox(children=(VBox(children=(RadioButtons(layout=Layout(width='100px'), options=('Local …

Loading dandiset metadata:   0%|          | 0/293 [00:00<?, ?it/s]

enterstream
enters3file
enterfsspec


/home/daksh/.local/lib/python3.10/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.6.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/daksh/.local/lib/python3.10/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.3.0 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/daksh/.local/lib/python3.10/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.3.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


[False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]
[False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False]
[False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, False]
[False, False, False, False, True, True, False, False, False, False, True, True, False, False, False, False]
[False, False, False, False, True, True, True, False, False, False, True, True, False, False, False, False]


In [42]:
app.tracelist

[ToggleButton(value=False, description='0', style=ToggleButtonStyle(text_color='#1f77b4'), tooltip='Description'),
 ToggleButton(value=False, description='1', style=ToggleButtonStyle(text_color='#ff7f0e'), tooltip='Description'),
 ToggleButton(value=False, description='2', style=ToggleButtonStyle(text_color='#2ca02c'), tooltip='Description'),
 ToggleButton(value=False, description='3', style=ToggleButtonStyle(text_color='#d62728'), tooltip='Description'),
 ToggleButton(value=True, description='4', style=ToggleButtonStyle(text_color='#9467bd'), tooltip='Description'),
 ToggleButton(value=True, description='5', style=ToggleButtonStyle(text_color='#8c564b'), tooltip='Description'),
 ToggleButton(value=True, description='6', style=ToggleButtonStyle(text_color='#e377c2'), tooltip='Description'),
 ToggleButton(value=False, description='7', style=ToggleButtonStyle(text_color='#7f7f7f'), tooltip='Description'),
 ToggleButton(value=False, description='8', style=ToggleButtonStyle(text_color='#bc

In [48]:
ls1 = [1,4]
ls2 = [1,2,3,4,4]
for i1,i2 in zip(ls1,ls2):
    print(str(i1)+" "+str(i2))

1 1
4 2


In [41]:
a = widgets.Checkbox( value=False, description='1', disabled=False, button_style='', # 'success', 'info', 'warning', 'danger' or '' 
                                                         tooltip='Description', icon='',color='red',background_color='yellow',width='30px'
                                                        )
a = a.style.text_color = app.colorList[1]

AttributeError: 'str' object has no attribute 'style'

In [68]:
def make_ts1(data,name="Trace"):
    return pynwb.base.TimeSeries(name=name,data=data,starting_time=0.,rate=16666.66,unit='seconds')

def make_ts1stim(nwbobj):
    data = np.array([app.nwb.stimulus[app.stimkeys[0]].data[:]])
    for i in range(1,len(app.stimkeys)):
        tempd = [app.nwb.stimulus[app.stimkeys[i]].data[:]]
        data = np.append(data,tempd,axis=0)
    return(self.make_ts1(data.T))

def make_ts1acq(nwbobj):
    data = np.array([app.nwb.acquisition[app.acqkeys[0]].data[:]])
    for i in range(1,len(app.acqkeys)):
        tempd = [app.nwb.acquisition[app.acqkeys[i]].data[:]]
        data = np.append(data,tempd,axis=0)
    return(make_ts1(data.T))

In [69]:
a
f = make_ts1acq(a)

In [70]:
f.data.shape

(52644, 16)

In [71]:
f.data.shape[1]

16

In [78]:
AlignMultiTraceTimeSeriesByTrialsVariable(time_series = f)

AttributeError: 'NoneType' object has no attribute 'trials'

In [8]:
display(app.widgets_panel.children)

()

    def make_ts1(self,data,name="Trace"):
        return pynwb.base.TimeSeries(name=name,data=data,starting_time=0.,rate=16666.66,unit='seconds')
    
    def make_ts1stim(self,nwbobj):
        data = np.array([app.nwb.stimulus[app.stimkeys[0]].data[:]])
        for i in range(1,len(app.stimkeys)):
            tempd = [app.nwb.stimulus[app.stimkeys[i]].data[:]]
            data = np.append(data,tempd,axis=0)
        return(self.make_ts1(data))
    
    def make_ts1acq(self,nwbobj):
        data = np.array([app.nwb.acquisition[app.acqkeys[0]].data[:]])
        for i in range(1,len(app.acqkeys)):
            tempd = [app.nwb.acquisition[app.acqkeys[i]].data[:]]
            data = np.append(data,tempd,axis=0)
        return(self.make_ts1(data))

In [38]:
a = widgets.ToggleButton( value=False, description='1', disabled=False, button_style='warning', # 'success', 'info', 'warning', 'danger' or '' 
                                                         tooltip='Description', icon='',color='red',background_color='yellow',width='30px'
                                                        )

In [35]:
a = widgets.Checkbox(value=False,description='1')

In [39]:
a.style.text_color = 'green'

In [40]:
display(a)

ToggleButton(value=False, button_style='warning', description='1', style=ToggleButtonStyle(text_color='green')…

In [13]:
a = [True,False]
b = [True,True]
True - False

1

In [14]:
app.fluorescence.out_fig

AttributeError: 'Dashboard2' object has no attribute 'fluorescence'

In [ ]:
app.fluorescence.out_fig.add_trace(go.Scatter(x=app.electrical.out_fig.data[0].x,y=app.nwb.stimulus[app.stimkeys[1]].data[:]))

In [ ]:
app.fluorescence.out_fig.data[0].visible = !(False)

In [ ]:
False

widgets.ToggleButton(
                 value=True,
                 description='Trace'+str(i),
                 disabled=False,
                 button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                 tooltip='Description',
                 icon='check')

In [ ]:
data = np.array([app.nwb.stimulus[app.stimkeys[0]].data[:]]).T
for i in range(1,len(app.stimkeys)):
            tempd = np.array([app.nwb.stimulus[app.stimkeys[i]].data[:]]).T
            data = np.append(data,tempd,axis=1)

In [ ]:
data.shape[1]

In [ ]:
data.shape

In [ ]:
SeparateTracesPlotlyWidget(
            timeseries=app.make_ts1(data),
                              foreign_time_window_controller=app.time_window_controller)

In [ ]:
l1 = [app.make_ts(app.nwb.stimulus[app.stimkeys[in1]]) for in1 in range(len(app.stimkeys))]

In [ ]:
MultiTimeSeriesWidget(l1)

In [ ]:
data

In [ ]:
data = [data,tempd]

In [ ]:
tempd

In [ ]:
app.stimkeys[15]

In [ ]:
def on_button_clicked(b):
    if isinstance(change["new"], float):
        print(b['new'])

In [ ]:
app.tracelist[0].observe(on_button_clicked)

In [ ]:
app.tracelist[0].__doc__

In [34]:
from pprint import pprint
pprint(dir(a.style))

['__annotations__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_active_widgets',
 '_add_notifiers',
 '_all_trait_default_generators',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_control_comm',
 '_cross_validation_lock',
 '_default_keys',
 '_descriptors',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_control_comm_msg',
 '_handle_custom_msg',
 '_handle_msg',
 '_holding_sync',
 '_instance_inits',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_observers',
 '

In [ ]:
app.electrical.out_fig.select_traces

In [ ]:
widgetlist
valuelist


def observe1(change):
    self.updatevals() # valuelist updated
    self.compute()

In [ ]:
def self.compute():

In [ ]:
a = [True,True]

a == [True for i in range(len(a))] #cond

In [ ]:
if cond1 | cond2
    addallt
else:
    addt where t

In [ ]:
app.nwb.acquisition

In [ ]:
def createAllTData(self):

In [ ]:
SingleTracePlotlyWidget(
            timeseries=self.make_ts(self.nwb.stimulus[self.stimkeys[in1]]),
                              foreign_time_window_controller=self.time_window_controller)
        for i in range(1,len(self.stimkeys)):
            self.fluorescence.out_fig.add_trace(self.nwb.stimulus[self.stimkeys[i]].data[:])